# Exercise 1: What can LLMs do and can't do

In [9]:
# set your open ai key in .env file (text file) as OPENAI_API_KEY=your-key
# your .env file should be in tthe same directory as this file
from dotenv import load_dotenv
_ = load_dotenv()

In [4]:
# pip install langchain langchain-openai

from langchain_openai import ChatOpenAI

# Create a simple chat model
llm = ChatOpenAI(model="gpt-4o-mini")  # or another chat model

# 1) LLMs are great: ask for some public facts → they tell you.
public_facts_prompt = (
    "List three well-known public facts about Florida in bullet points."
)
public_facts_response = llm.invoke(public_facts_prompt)
print("=== Public facts example ===")
print(public_facts_response.content)
print()

# 2) Write a customer email → done.
customer_email_prompt = (
    "Write a short, polite email to a customer explaining that their order "
    "shipment is delayed by two days, and reassure them."
)
customer_email_response = llm.invoke(customer_email_prompt)
print("=== Customer email example ===")
print(customer_email_response.content)

=== Public facts example ===
- **Home to the Everglades**: Florida is known for the Everglades National Park, a vast and unique wetland ecosystem that is a UNESCO World Heritage Site and is home to diverse wildlife, including endangered species like the Florida panther.

- **Tourist Destination**: Florida is one of the most popular tourist destinations in the United States, attracting millions of visitors each year to attractions such as Walt Disney World, Universal Studios, and its beautiful beaches along the Atlantic and Gulf coasts.

- **Hurricanes**: Florida is frequently impacted by hurricanes and tropical storms due to its location in the southeastern United States, with a hurricane season that typically runs from June to November. The state's geography makes it particularly vulnerable to these natural disasters.

=== Customer email example ===
Subject: Update on Your Order Shipment

Dear [Customer's Name],

I hope this message finds you well. I wanted to inform you that there ha

## Illustrate what LLMs can't do by themselves

In [5]:
# pip install langchain langchain-openai
# Make sure OPENAI_API_KEY is set in your environment.

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")  # any OpenAI chat model

# 1) Ask as if it can check Florida weather and book tickets
weather_and_booking_prompt = """
You are connected to live weather data and a travel booking system.

Task:
1) Check the weather forecast for Miami, Florida for next week.
2) Choose the best 3-day window with good weather.
3) Book round-trip flight tickets from New York to Miami for those dates.
4) Confirm the booking details (dates, airline, times, and price).

If you cannot actually do these steps, explain clearly what you *can* do instead.
"""

weather_response = llm.invoke(weather_and_booking_prompt)
print("=== Florida weather and booking attempt ===")
print(weather_response.content)
print()

# 2) Ask as if it can read past transactions and customer history
customer_history_prompt = """
You have direct access to our live customer database and transaction history.

Customer ID: 12345

Task:
1) Read this customer's past 12 months of purchases and support tickets.
2) Summarize their purchase and support history.
3) Then write a personalized, meaningful email that references specific past orders
   and issues to thank them and offer a relevant promotion.

If you cannot actually read real data, explain clearly what you *can* do instead.
"""

customer_response = llm.invoke(customer_history_prompt)
print("=== Customer history and meaningful email attempt ===")
print(customer_response.content)

=== Florida weather and booking attempt ===
I apologize for any confusion, but I don't have the ability to access live weather data or make travel bookings. However, I can guide you on how to check the weather forecast and book flights:

### Step 1: Check the Weather Forecast
- You can use reliable weather websites like Weather.com, AccuWeather, or a weather app on your smartphone. Simply enter "Miami, Florida" and look for the forecast for next week. Pay attention to factors like temperature, precipitation, and wind speed to find the best days for your trip.

### Step 2: Identify the Best 3-Day Window
- Look for days with the best weather conditions – generally, this means mild temperatures (not too hot), little to no chance of rain, and calm winds. Typically, mid-week days might be less crowded and may offer better rates.

### Step 3: Book Round-Trip Flights
- Once you have chosen your dates, head to popular flight booking websites like Expedia, Kayak, Google Flights, or directly to 

In [7]:
## Illustrate an LLMs that is connected to an external Weather API.

In [11]:
# pip install langchain langchain-openai
# Make sure OPENAI_API_KEY is set in your environment.

import requests
from langchain_openai import ChatOpenAI
from langchain.tools import tool

# Your weather API key (e.g., OpenWeatherMap)
WEATHER_API_KEY = "7b75531d0f1549cca18220335251712"  # Or load from .env


# Define a tool the LLM can use
@tool
def get_weather(city: str) -> str:
    """Get current weather for a city."""
    url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={WEATHER_API_KEY}&units=imperial"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        temp = data["main"]["temp"]
        description = data["weather"][0]["description"]
        return f"Weather in {city}: {temp}°F, {description}"
    return f"Could not fetch weather for {city}"

llm = ChatOpenAI(model="gpt-4o-mini")  # any OpenAI chat model
llm_with_tools = llm.bind_tools([get_weather])

# 1) Ask as if it can check Florida weather and book tickets
weather_and_booking_prompt = """
You are connected to live weather data and a travel booking system.

Task:
1) Check the weather forecast for Miami, Florida for next week.
2) Choose the best 3-day window with good weather.
3) Book round-trip flight tickets from New York to Miami for those dates.
4) Confirm the booking details (dates, airline, times, and price).

If you cannot actually do these steps, explain clearly what you *can* do instead.
"""

weather_response = llm.invoke(weather_and_booking_prompt)
print("=== Florida weather and booking attempt ===")
response = llm_with_tools.invoke("What's the weather in Miami, Florida?")
print(response)

# 2) Ask as if it can read past transactions and customer history
customer_history_prompt = """
You have direct access to our live customer database and transaction history.

Customer ID: 12345

Task:
1) Read this customer's past 12 months of purchases and support tickets.
2) Summarize their purchase and support history.
3) Then write a personalized, meaningful email that references specific past orders
   and issues to thank them and offer a relevant promotion.

If you cannot actually read real data, explain clearly what you *can* do instead.
"""

customer_response = llm.invoke(customer_history_prompt)
print("=== Customer history and meaningful email attempt ===")
print(customer_response.content)

=== Florida weather and booking attempt ===
content='' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 52, 'total_tokens': 68, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_1590f93f9d', 'id': 'chatcmpl-D4FrmXV0BDalHdJZxNeq4NXnkp7dk', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None} id='lc_run--019c16a8-9a91-7fc2-82be-77d9bc8b2d1e-0' tool_calls=[{'name': 'get_weather', 'args': {'city': 'Miami, Florida'}, 'id': 'call_4HSnw7Y8DPICih4w93Fc1jCT', 'type': 'tool_call'}] invalid_tool_calls=[] usage_metadata={'input_tokens': 52, 'output_tokens': 16, 'total_tokens': 68, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio':

Test a Weather Call

In [3]:
# set your open ai key in .env file (text file) as OPENAI_API_KEY=your-key
# your .env file should be in tthe same directory as this file
from dotenv import load_dotenv
_ = load_dotenv()

# pip install langchain langchain-openai
# Make sure OPENAI_API_KEY is set in your environment.

import requests
from langchain_openai import ChatOpenAI
from langchain.tools import tool

# Your weather API key (e.g., OpenWeatherMap)
WEATHER_API_KEY = "7b75531d0f1549cca18220335251712"  # Or load from .env


# Define a tool the LLM can use
@tool
def get_weather(city: str) -> str:
    """Get current weather for a city."""
    url = f"http://api.weatherapi.com/v1/current.json?key={WEATHER_API_KEY}&q={city}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        temp = data["current"]["temp_f"]
        description = data["current"]["condition"]["text"]
        return f"Weather in {city}: {temp}°F, {description}"
    return f"Could not fetch weather for {city}"

llm = ChatOpenAI(model="gpt-4o-mini")  # any OpenAI chat model
llm_with_tools = llm.bind_tools([get_weather])


response = llm_with_tools.invoke("What's the weather in Miami, Florida?")
#print(response)

# Check if the LLM wants to call a tool
if response.tool_calls:
    for tool_call in response.tool_calls:
        if tool_call['name'] == 'get_weather':
            # Execute the tool with the arguments the LLM provided
            city = tool_call['args']['city']
            weather_result = get_weather.invoke(city)
            print(weather_result)
else:
   print(response.content)

Weather in Miami, Florida: 57.9°F, Overcast
